In [22]:
import datetime
import logging
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from collections import Counter
import numpy as np
from sklearn.metrics import silhouette_score


def configure_logging():
    """
    配置日志记录
    """
    logging.basicConfig(filename='processing_log.log', level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s')


def log_message(message):
    """
    记录日志信息并打印
    """
    logging.info(message)
    print(message)


def read_csv_data(file_path):
    """
    从CSV文件中读取数据并获取文本列和分区列（假设列名固定）
    :param file_path: CSV文件路径
    :return: 包含数据的DataFrame、文本列名和分区列名
    """
    df = pd.read_csv(file_path)
    text_column = '所有评论和弹幕'
    partition_column = '分区'
    return df, text_column, partition_column


def extract_tfidf_features(df, text_column):
    """
    提取TF - IDF特征
    :param df: 包含文本的DataFrame
    :param text_column: 文本所在的列名
    :return: TF - IDF矩阵和特征词汇表
    """
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df[text_column])
    feature_names = vectorizer.get_feature_names_out()
    log_message(f'TF - IDF特征提取完成，特征词汇数量: {len(feature_names)}')
    return tfidf_matrix, feature_names


def perform_clustering(tfidf_matrix):
    """
    进行K - Means聚类并返回聚类标签
    :param tfidf_matrix: TF - IDF矩阵
    :return: 聚类标签
    """
    silhouette_scores = []
    # 尝试不同的聚类数量，这里假设范围是从 2 到 10
    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(tfidf_matrix)
        labels = kmeans.labels_
        silhouette_avg = silhouette_score(tfidf_matrix, labels)
        silhouette_scores.append(silhouette_avg)
    best_k = np.argmax(silhouette_scores) + 2  # 因为索引从 0 开始，所以要加 2
    log_message(f'最佳聚类数量为: {best_k}')
    kmeans = KMeans(n_clusters=best_k)
    kmeans.fit(tfidf_matrix)
    return kmeans.labels_


def analyze_clusters(df, cluster_labels, partition_column, cluster_column='聚类'):
    """
    分析各聚类的文本数量及常用词汇与分区的关系
    :param df: 包含新闻数据的DataFrame
    :param cluster_labels: 聚类标签
    :param partition_column: 分区列名
    :param cluster_column: 聚类列名
    """
    df[cluster_column] = cluster_labels
    unique_clusters = df[cluster_column].unique()
    unique_partitions = df[partition_column].unique()

    for cluster in unique_clusters:
        cluster_data = df[df[cluster_column] == cluster]
        log_message(f'聚类 {cluster} 的文本数量: {len(cluster_data)}')

        for partition in unique_partitions:
            partition_data = cluster_data[cluster_data[partition_column] == partition]
            log_message(f'分区 {partition} 中聚类 {cluster} 的文本数量: {len(partition_data)}')

        # 分析常用词汇
        cluster_content = ' '.join(cluster_data['所有评论和弹幕'])
        words = cluster_content.split()
        word_freq = Counter(words)
        log_message(f'聚类 {cluster} 的常用词汇及词频前20: {word_freq.most_common(20)}')


def generate_cross_tab(df, partition_column, cluster_column):
    """
    生成交叉表验证聚类效果
    :param df: 包含数据的DataFrame
    :param partition_column: 分区列名
    :param cluster_column: 聚类列名
    :return: 交叉表
    """
    cross_tab = pd.crosstab(df[partition_column], df[cluster_column], margins=True, margins_name='合计')
    log_message('交叉表生成完成')
    log_message(cross_tab)
    return cross_tab


def main():
    file_path = './output_withdanmaku_csv.csv'
    me = 'by [22251109115] [严秋实]'

    configure_logging()
    log_message('1. 从CSV文件中读取数据并自动确定列名' + me)
    print('开始时间：' + str(datetime.datetime.now()))
    df, text_column, partition_column = read_csv_data(file_path)
    log_message(f'自动获取的文本列名: {text_column}')
    log_message(f'自动获取的分区列名: {partition_column}')
    print('数据基本信息：')
    df.info()
    print('数据前几行：')
    print(df.head())
    print('结束时间：' + str(datetime.datetime.now()))

    log_message('2. 抽取词向量特征' + me)
    print('开始时间：' + str(datetime.datetime.now()))
    tfidf_matrix, feature_names = extract_tfidf_features(df, text_column)
    print('TF - IDF矩阵形状:')
    print(tfidf_matrix.shape)
    print('特征词汇表:')
    print(feature_names)
    print('结束时间：' + str(datetime.datetime.now()))

    log_message('3. 进行聚类' + me)
    print('开始时间：' + str(datetime.datetime.now()))
    cluster_labels = perform_clustering(tfidf_matrix)
    log_message('聚类完成')
    print('添加聚类标签后的DataFrame部分内容:')
    print(df.head().assign(聚类=cluster_labels[:5]))
    print('结束时间：' + str(datetime.datetime.now()))

    log_message('4. 分析各聚类的文本数量及常用词汇与分区的关系' + me)
    print('开始时间：' + str(datetime.datetime.now()))
    analyze_clusters(df, cluster_labels, partition_column)
    print('结束时间：' + str(datetime.datetime.now()))

    log_message('5. 生成交叉表验证聚类效果' + me)
    print('开始时间：' + str(datetime.datetime.now()))
    cross_tab = generate_cross_tab(df, partition_column, '聚类')
    print('结束时间：' + str(datetime.datetime.now()))


if __name__ == "__main__":
    main()

1. 从CSV文件中读取数据并自动确定列名by [22251109115] [严秋实]
开始时间：2024-12-30 23:25:05.225775
自动获取的文本列名: 所有评论和弹幕
自动获取的分区列名: 分区
数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   序号       300 non-null    int64 
 1   分区       300 non-null    object
 2   视频 ID    300 non-null    object
 3   所有评论和弹幕  300 non-null    object
dtypes: int64(1), object(3)
memory usage: 9.5+ KB
数据前几行：
   序号     分区         视频 ID                                            所有评论和弹幕
0   1  douga  BV16mBMY4EsZ  补充 几个 漏掉 绝望 跳出 舞步 吹牛 这是 开过 忆昔 当年 泪不干 六七 记忆 碎片 ...
1   2  douga  BV1nvBLYUEzn  名字 如雷贯耳 看过 白箱 说实话 从业者 排斥 接触 业界 相关 作品 从业者 难免 一种...
2   3  douga  BV1VhBNYrExc  深夜 视频 先存 明天 翻书 好久 复习 正当防卫 内容 正当防卫 飞车 抢劫 暴力 压制 ...
3   4  douga  BV1pjq9YpEEG  麻烦 耽误 一点 时间 生日 不想 耽误 视频 很棒 三连哩 动画 上班 一集 幼教 动画 ...
4   5  douga  BV16HqdYdELG  浅浅的 催个 充电 支持 希望 别跑 嗑瓜子 小说 故事 故事 结局 温馨 作者 文笔 看着...
结束时间：2024-12-30 23:25:05.417625
